# Let's start some basic analytics

In [1]:
import sys
import sqlite3
from collections import Counter

import pandas as pd

sys.path.append("..")

import src.utils as utils


In [2]:
db_path = utils.root_path / "data" / "data.db"
db_path.exists()

True

In [3]:
conn = sqlite3.connect(str(db_path))

In [4]:
query = """SELECT forecasts.CITY, 
        date(FORECAST_DATE) as forecast_date, 
        PLATFORM, 
        forecasts.TEMPERATURE as forecast_temperature,
        date(DATE) as observed_date,
        observed_temp.TEMPERATURE as observed_temperature
 FROM forecasts
LEFT JOIN observed_temp
ON forecasts.city = observed_temp.city
AND date(forecasts.forecast_date) = date(observed_temp.date);
"""


In [5]:
df = pd.read_sql_query(query, conn)
conn.close()
df

,CITY,forecast_date,PLATFORM,forecast_temperature,observed_date,observed_temperature
0,CHI,2022-11-05,TWC,65,2022-11-05,71.0
1,NYC,2022-11-05,TWC,73,2022-11-05,76.0
2,NYC,2022-11-05,Accuweather,73,2022-11-05,76.0
3,CHI,2022-11-05,Accuweather,63,2022-11-05,71.0
4,NYC,2022-11-05,NWS,72,2022-11-05,76.0
...,...,...,...,...,...,...
208,CHI,2022-11-15,Accuweather,40,None,NaN
209,NYC,2022-11-15,NWS,46,None,NaN
210,CHI,2022-11-15,NWS,38,None,NaN
211,NYC,2022-11-15,foreca,45,None,NaN


In [6]:
df = df.dropna(subset=['observed_temperature', "forecast_temperature"])
df

,CITY,forecast_date,PLATFORM,forecast_temperature,observed_date,observed_temperature
0,CHI,2022-11-05,TWC,65,2022-11-05,71.0
1,NYC,2022-11-05,TWC,73,2022-11-05,76.0
2,NYC,2022-11-05,Accuweather,73,2022-11-05,76.0
3,CHI,2022-11-05,Accuweather,63,2022-11-05,71.0
4,NYC,2022-11-05,NWS,72,2022-11-05,76.0
...,...,...,...,...,...,...
192,CHI,2022-11-13,Accuweather,40,2022-11-13,53.0
193,NYC,2022-11-13,NWS,52,2022-11-13,61.0
194,CHI,2022-11-13,NWS,40,2022-11-13,53.0
195,NYC,2022-11-13,foreca,57,2022-11-13,61.0


In [7]:
# this should be nothing, but it's worth a check
df[df["forecast_temperature"] == '']

,CITY,forecast_date,PLATFORM,forecast_temperature,observed_date,observed_temperature


In [8]:
df["observed_temperature"] = df["observed_temperature"].astype(int)
df["forecast_temperature"] = df["forecast_temperature"].astype(int)
df

/tmp/ipykernel_13634/2382561297.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["observed_temperature"] = df["observed_temperature"].astype(int)
/tmp/ipykernel_13634/2382561297.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["forecast_temperature"] = df["forecast_temperature"].astype(int)


,CITY,forecast_date,PLATFORM,forecast_temperature,observed_date,observed_temperature
0,CHI,2022-11-05,TWC,65,2022-11-05,71
1,NYC,2022-11-05,TWC,73,2022-11-05,76
2,NYC,2022-11-05,Accuweather,73,2022-11-05,76
3,CHI,2022-11-05,Accuweather,63,2022-11-05,71
4,NYC,2022-11-05,NWS,72,2022-11-05,76
...,...,...,...,...,...,...
192,CHI,2022-11-13,Accuweather,40,2022-11-13,53
193,NYC,2022-11-13,NWS,52,2022-11-13,61
194,CHI,2022-11-13,NWS,40,2022-11-13,53
195,NYC,2022-11-13,foreca,57,2022-11-13,61


In [9]:
df["error"] = df["observed_temperature"] -  df["forecast_temperature"] 
df

/tmp/ipykernel_13634/3409431157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["error"] = df["observed_temperature"] -  df["forecast_temperature"]


,CITY,forecast_date,PLATFORM,forecast_temperature,observed_date,observed_temperature,error
0,CHI,2022-11-05,TWC,65,2022-11-05,71,6
1,NYC,2022-11-05,TWC,73,2022-11-05,76,3
2,NYC,2022-11-05,Accuweather,73,2022-11-05,76,3
3,CHI,2022-11-05,Accuweather,63,2022-11-05,71,8
4,NYC,2022-11-05,NWS,72,2022-11-05,76,4
...,...,...,...,...,...,...,...
192,CHI,2022-11-13,Accuweather,40,2022-11-13,53,13
193,NYC,2022-11-13,NWS,52,2022-11-13,61,9
194,CHI,2022-11-13,NWS,40,2022-11-13,53,13
195,NYC,2022-11-13,foreca,57,2022-11-13,61,4


In [10]:
df_chi = df[df["CITY"]=='CHI']

In [11]:
errors = df_chi[["PLATFORM", "error"]].groupby('PLATFORM')["error"].value_counts()
errors

PLATFORM     error
Accuweather   0       4
              2       4
             -4       2
              1       2
              3       2
              4       2
              5       1
              6       1
              8       1
              9       1
              13      1
NWS           3       6
              4       5
              5       2
              10      2
             -2       1
              0       1
              1       1
              2       1
              9       1
              13      1
TWC           1       6
              0       5
             -2       2
              2       2
             -5       1
              3       1
              5       1
              6       1
              8       1
              12      1
foreca        0       5
              4       3
             -3       2
              2       2
              3       2
              1       1
              5       1
              8       1
              9       1
              14     

In [12]:
accu_error = errors["Accuweather"]
accu_error

error
 0     4
 2     4
-4     2
 1     2
 3     2
 4     2
 5     1
 6     1
 8     1
 9     1
 13    1
Name: error, dtype: int64

In [13]:
errors_dict = errors["Accuweather"].to_dict()
errors_dict

{0: 4, 2: 4, -4: 2, 1: 2, 3: 2, 4: 2, 5: 1, 6: 1, 8: 1, 9: 1, 13: 1}

## Mean

Okay I want to see what the mean looks like as well

In [14]:
means = df_chi.groupby("forecast_date")[["forecast_temperature", "observed_temperature"]].mean()
means

,forecast_temperature,observed_temperature
forecast_date,,
2022-10-15,54.500000,55.0
2022-10-16,55.750000,57.0
2022-10-17,42.000000,45.0
2022-10-18,46.500000,50.0
2022-10-23,78.250000,81.0
2022-10-24,76.333333,79.0
2022-10-25,64.333333,71.0
2022-10-26,56.500000,57.0
2022-10-27,56.250000,54.0


In [15]:
means["f_t_round"] = means["forecast_temperature"].round().astype(int)
means["error"] = means["observed_temperature"] - means["f_t_round"]

In [16]:
means["error"].mean()

3.0476190476190474

In [17]:
means["error"].abs().mean()

3.619047619047619

### Metrics

Okay so I guess we want to get the mean and stdev?

In [18]:
df_chi_accu = df_chi[df_chi["PLATFORM"] == "Accuweather"]
df_chi_accu

,CITY,forecast_date,PLATFORM,forecast_temperature,observed_date,observed_temperature,error
3,CHI,2022-11-05,Accuweather,63,2022-11-05,71,8
11,CHI,2022-11-06,Accuweather,63,2022-11-06,64,1
19,CHI,2022-11-07,Accuweather,56,2022-11-07,58,2
27,CHI,2022-11-08,Accuweather,58,2022-11-08,54,-4
35,CHI,2022-11-09,Accuweather,68,2022-11-09,70,2
124,CHI,2022-10-31,Accuweather,61,2022-10-31,61,0
128,CHI,2022-10-30,Accuweather,63,2022-10-30,67,4
132,CHI,2022-10-29,Accuweather,65,2022-10-29,67,2
136,CHI,2022-10-28,Accuweather,61,2022-10-28,61,0
140,CHI,2022-10-27,Accuweather,58,2022-10-27,54,-4


In [19]:
df_chi_accu["error"].mean()
df_chi_accu["error"].abs().mean()

3.4761904761904763

In [20]:
def metrics(df, platform, city):
    df = df.copy()
    
    df = df[df["CITY"]== city]
    df = df[df["PLATFORM"] == platform]
    
    mean = df["error"].mean()
    abs_mean = df["error"].abs().mean()
    
    print(f'Mean Error for {platform} in {city}: {mean}')
    print(f'Mean Absolute Error for {platform} in {city}: {abs_mean}')
    
metrics(df, "foreca", "CHI")

Mean Error for foreca in CHI: 2.789473684210526
Mean Absolute Error for foreca in CHI: 3.4210526315789473


In [21]:
for city in set(df["CITY"]):
    
    for platform in set(df["PLATFORM"]):
        
        metrics(df, platform, city)
        print("\n")

Mean Error for foreca in NYC: 1.1
Mean Absolute Error for foreca in NYC: 2.3


Mean Error for Accuweather in NYC: 0.14285714285714285
Mean Absolute Error for Accuweather in NYC: 1.8571428571428572


Mean Error for TWC in NYC: 1.0476190476190477
Mean Absolute Error for TWC in NYC: 1.5238095238095237


Mean Error for NWS in NYC: 0.5238095238095238
Mean Absolute Error for NWS in NYC: 2.142857142857143


Mean Error for foreca in CHI: 2.789473684210526
Mean Absolute Error for foreca in CHI: 3.4210526315789473


Mean Error for Accuweather in CHI: 2.7142857142857144
Mean Absolute Error for Accuweather in CHI: 3.4761904761904763


Mean Error for TWC in CHI: 1.6666666666666667
Mean Absolute Error for TWC in CHI: 2.5238095238095237


Mean Error for NWS in CHI: 4.333333333333333
Mean Absolute Error for NWS in CHI: 4.523809523809524




### Get Odds

Okay this is going to be super crazy but whatever

Heres' a good start: https://stackoverflow.com/questions/38711541/how-to-compute-the-probability-of-a-value-given-a-list-of-samples-from-a-distrib

In [22]:
def odds(df, platform, city):
    df = df.copy()
    df = df[df["CITY"]== city]
    df = df[df["PLATFORM"] == platform]
    
    counts = df["error"].value_counts()
    total = counts.sum()
    
    
    
    return counts / total

error_odds = odds(df, "foreca", "CHI")
error_odds

 0     0.263158
 4     0.157895
-3     0.105263
 3     0.105263
 2     0.105263
 5     0.052632
 1     0.052632
 8     0.052632
 9     0.052632
 14    0.052632
Name: error, dtype: float64

### And now given that table

We want to be able to say, what is the probability of it being 71 deg when the forecast is 72.

In [23]:
def probability_blaster(forecast, threshold, operator, odds):
    diff = forecast - threshold
    
    if operator == "lt":
        new_odds = odds[odds.keys() < diff]
    
    return new_odds.sum()
    
probability_blaster(71, 72, "lt", error_odds)

0.10526315789473684

### Here is the next step!!!!

So let's say you wanted to have an ensemble. You can't just get the probabilities from each of them. Just think bout it it now that you have this.

So what you have to do is get the numbers and the counts for each platform and then go from there.

NOW! this only matters if it's a different number of observations. This is worth investigating.

df_chi[["PLATFORM", "error"]].groupby('PLATFORM')["error"].value_counts()

In [24]:
df[df["CITY"] == "CHI"]

,CITY,forecast_date,PLATFORM,forecast_temperature,observed_date,observed_temperature,error
0,CHI,2022-11-05,TWC,65,2022-11-05,71,6
3,CHI,2022-11-05,Accuweather,63,2022-11-05,71,8
5,CHI,2022-11-05,NWS,61,2022-11-05,71,10
7,CHI,2022-11-05,foreca,66,2022-11-05,71,5
8,CHI,2022-11-06,TWC,64,2022-11-06,64,0
...,...,...,...,...,...,...,...
188,CHI,2022-11-12,foreca,39,2022-11-12,48,9
189,CHI,2022-11-13,TWC,41,2022-11-13,53,12
192,CHI,2022-11-13,Accuweather,40,2022-11-13,53,13
194,CHI,2022-11-13,NWS,40,2022-11-13,53,13


In [25]:
def error_counts(df, city):
    df = df[df["CITY"] == city].copy()
    value_counts = df[["PLATFORM", "error"]].groupby('PLATFORM')["error"].value_counts().rename("error_count")
    value_counts = value_counts.to_frame()
    value_counts = value_counts.reset_index(level='error')
    
    return value_counts

df_co = error_counts(df, "CHI")
df_co

,error,error_count
PLATFORM,,
Accuweather,0,4
Accuweather,2,4
Accuweather,-4,2
Accuweather,1,2
Accuweather,3,2
Accuweather,4,2
Accuweather,5,1
Accuweather,6,1
Accuweather,8,1


In [26]:
def error_samples(df, city):
    df = df[df["CITY"] == city].copy()
    counts = df[["PLATFORM", "error"]].groupby("PLATFORM").count()
    counts.columns=["samples"]
    
    return counts

error_samples(df, "CHI")

,samples
PLATFORM,
Accuweather,21
NWS,21
TWC,21
foreca,19


error = df["observed_temperature"] -  df["forecast_temperature"] 

In [27]:
def combined_odds(df, city, forecast, operator, threshold):
    
    errors = error_counts(df, city)
    error_to_predict = threshold - forecast
    
    # trim the erros based on the operator
    if operator == "lt=":
        errors = errors[errors["error"] <= error_to_predict]
    
    summed_errors = errors.groupby("PLATFORM")["error_count"].sum().to_frame()
    
    samples = error_samples(df, city)
    
    fin = samples.join(summed_errors).fillna(0)
    
    total_errors = fin["error_count"].sum()
    total_samples = fin["samples"].sum()
    
    probability = total_errors / total_samples
    
    fin["probability"] = fin["error_count"] / fin["samples"]
    
    return fin, probability

k = combined_odds(df, "CHI", 75, "lt=", 71)
k

(             samples  error_count  probability
 PLATFORM                                      
 Accuweather       21          2.0     0.095238
 NWS               21          0.0     0.000000
 TWC               21          1.0     0.047619
 foreca            19          0.0     0.000000,
 0.036585365853658534)

## Except that's not it, is it

Now we have to

1) Get the forecast for each platoform

2) Trim each one based on that

3) THEN do what we've done above

In [28]:
query = """SELECT * FROM forecasts
    WHERE date(FORECAST_DATE) =
        (SELECT date(FORECAST_DATE) FROM forecasts
        ORDER BY date(FORECAST_DATE) desc
        LIMIT 1);"""

In [29]:
conn = sqlite3.connect(str(db_path))

In [30]:
df_f = pd.read_sql_query(query, conn)
conn.close()
df_f

,ID,CITY,FORECAST_DATE,TIME_OF_FORECAST,PLATFORM,TEMPERATURE
0,None,CHI,2022-11-15 17:28:33.537346,2022-11-14 17:28:33.537341,TWC,40
1,None,NYC,2022-11-15 17:28:33.537346,2022-11-14 17:28:33.537341,TWC,45
2,None,NYC,2022-11-15 17:28:33.537346,2022-11-14 17:28:33.537341,Accuweather,46
3,None,CHI,2022-11-15 17:28:33.537346,2022-11-14 17:28:33.537341,Accuweather,40
4,None,NYC,2022-11-15 17:28:33.537346,2022-11-14 17:28:33.537341,NWS,46
5,None,CHI,2022-11-15 17:28:33.537346,2022-11-14 17:28:33.537341,NWS,38
6,None,NYC,2022-11-15 17:28:33.537346,2022-11-14 17:28:33.537341,foreca,45
7,None,CHI,2022-11-15 17:28:33.537346,2022-11-14 17:28:33.537341,foreca,37


In [35]:
def trim_errors(df, platform, city, error, operator):
    df = df.copy()
    return df

In [36]:
def errors_by_platform(df, city, platform, forecast, threshold):
    error = threshold - forecast
    k = trim_errors(df, platform, city,  error)
    
    return k
    
errors_by_platform(df, "CHI", "TWC", 40, 38)

,CITY,forecast_date,PLATFORM,forecast_temperature,observed_date,observed_temperature,error
0,CHI,2022-11-05,TWC,65,2022-11-05,71,6
1,NYC,2022-11-05,TWC,73,2022-11-05,76,3
2,NYC,2022-11-05,Accuweather,73,2022-11-05,76,3
3,CHI,2022-11-05,Accuweather,63,2022-11-05,71,8
4,NYC,2022-11-05,NWS,72,2022-11-05,76,4
...,...,...,...,...,...,...,...
192,CHI,2022-11-13,Accuweather,40,2022-11-13,53,13
193,NYC,2022-11-13,NWS,52,2022-11-13,61,9
194,CHI,2022-11-13,NWS,40,2022-11-13,53,13
195,NYC,2022-11-13,foreca,57,2022-11-13,61,4
